In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import concurrent.futures
from typing import List

In [2]:
cleanLOBFolder = '/media/ak/Data/CleanLOBData'
interestRateFuturesFolder = '/media/ak/Data/InterestRateFuturesData'
irFFquotesTrades = os.path.join(interestRateFuturesFolder, 'QuotesTrades')
rawData = os.path.join(interestRateFuturesFolder, 'Raw Data')
cleanLOBSymbols = ['DU1', 'FB1', 'FV1', 'G', 'JB1', 'KE1', 'OE1', 'RX1']
irfSymbols = ['DU1', 'FB1', 'RX1', 'SI1', 'TY1', 'US1', 'UST2y', 'VXX', 'XM1', 'YM1']

In [3]:
os.listdir(cleanLOBFolder)

['DU1', 'FB1', 'FV1', 'G', 'JB1', 'KE1', 'OE1', 'RX1']

In [4]:
US1 = os.path.join(irFFquotesTrades, 'US1')
quotesPath = os.path.join(US1, 'US1_Comdty_quotes')
tradesPath = os.path.join(US1, 'US1_Comdty_trades')

In [5]:
quotes = os.listdir(quotesPath)
trades = os.listdir(tradesPath)

In [6]:
difference_result = [item for item in quotes if item not in trades]

print(difference_result)

[]


In [7]:
file = quotes[30]
quoteFileLoc = os.path.join(quotesPath, file)
tradeFileLoc = os.path.join(tradesPath, file)

In [8]:
quoteFileLoc, tradeFileLoc

('/media/ak/Data/InterestRateFuturesData/QuotesTrades/US1/US1_Comdty_quotes/20180820.csv',
 '/media/ak/Data/InterestRateFuturesData/QuotesTrades/US1/US1_Comdty_trades/20180820.csv')

In [9]:
quotes_df_unproc = pd.read_csv(quoteFileLoc)
trades_df = pd.read_csv(tradeFileLoc)

In [10]:

directory = os.path.join('/media/ak/Data/InterestRateFuturesData/ReconstructedLOB/', 'XM1')
file = os.path.join('/media/ak/Data/InterestRateFuturesData/XM1_cleanLOBLocs.pkl')

In [11]:
os.listdir(interestRateFuturesFolder)

['DU1_AllDataFrames.pkl',
 'DU1_cleanLOBLocs.pkl',
 'EventClocksFiles',
 'QuotesTrades',
 'Raw Data',
 'ReconstructedLOB',
 'TSData',
 'US1_AllDataFrames.pkl',
 'US1_cleanLOBLocs.pkl',
 'XM1_AllDataFrames.pkl',
 'XM1_cleanLOBLocs.pkl',
 'YM1_AllDataFrames.pkl',
 'YM1_cleanLOBLocs.pkl']

In [12]:
lobsdictXM = pd.read_pickle('/media/ak/Data/InterestRateFuturesData/XM1_cleanLOBLocs.pkl')
lobsdictYM = pd.read_pickle('/media/ak/Data/InterestRateFuturesData/YM1_cleanLOBLocs.pkl')

In [13]:
keysYM = list(lobsdictYM.keys())
keysXM = list(lobsdictXM.keys())

In [14]:
testXM = pd.read_pickle(lobsdictXM[keysXM[20]])
testYM = pd.read_pickle(lobsdictYM[keysYM[20]])

In [ ]:
import numpy as np
import pandas as pd

def garman_klass(df, clean=True):
    df['period'] = (pd.to_datetime(df[('time', 'last')]) - pd.to_datetime(df[('time', 'first')])).dt.total_seconds() / (24 * 60 * 60)
    
    # Use np.where to avoid division by zero in log calculation
    df['log_hl'] = np.where((df[('MicroPrice', 'high')] != df[('MicroPrice', 'low')]), np.log(df[('MicroPrice', 'high')] / df[('MicroPrice', 'low')]), 0)
    df['log_cc'] = np.where((df[('MicroPrice', 'close')] != df[('MicroPrice', 'open')]), np.log(df[('MicroPrice', 'close')] / df[('MicroPrice', 'open')]), 0)
    df['garman_klass_vol'] = np.sqrt(0.5 * df['log_hl']**2 - (2*np.log(2) - 1) * df['log_cc']**2) / np.sqrt(df['period'])

    if clean:
        df = df.dropna()
    return df


In [16]:
testXM

,time,estimated_arrival_time,TradePrice,TradedVolume,NoOfTrades,arrival_rate,AverageVolumePerTrade,date,CCYTradedVolume,type,...,RunningNoOfTrades,RunningCumlVolume,RunningCumlNoOfTrades,MicroPrice,BASpread,QuoteSize,MidPrice,OrderImbalance,ExpandingMeanRelativeTickVolume,MeanRelativeTickVolume
1826250,2018-05-09 00:00:00,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-09,NaN,BID,...,1.0,1.064112e+07,11751.0,97.213942,0.005,8311.0,97.2125,0.576946,NaN,0.0
66838,2018-05-09 00:00:00,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-09,NaN,ASK,...,1.0,1.064112e+07,11751.0,97.213940,0.005,8316.0,97.2125,0.575998,NaN,0.0
83220,2018-05-09 00:00:00,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-09,NaN,BID,...,1.0,1.064112e+07,11751.0,97.213940,0.005,8316.0,97.2125,0.575998,NaN,0.0
1809030,2018-05-09 00:00:00,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-09,NaN,ASK,...,1.0,1.064112e+07,11751.0,97.213940,0.005,8316.0,97.2125,0.575998,NaN,0.0
83221,2018-05-09 00:00:01,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-09,NaN,BID,...,1.0,1.064112e+07,11751.0,97.213248,0.005,5031.0,97.2125,0.299145,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74641,2018-05-09 23:59:21,1726-01-29 00:11:59.145224192,97.19,1.0,1.0,0.000012,1.0,2018-05-09,97.19,BID,...,1.0,1.184757e+07,11389.0,97.188430,0.005,1825.0,97.1875,0.372055,0.0,0.0
74642,2018-05-09 23:59:22,1726-01-29 00:11:59.145224192,97.19,1.0,1.0,0.000012,1.0,2018-05-09,97.19,BID,...,1.0,1.184757e+07,11389.0,97.189070,0.005,3082.0,97.1875,0.628164,0.0,0.0
74643,2018-05-09 23:59:28,1726-01-29 00:11:59.145224192,97.19,1.0,1.0,0.000012,1.0,2018-05-09,97.19,BID,...,1.0,1.184757e+07,11389.0,97.188429,0.005,1824.0,97.1875,0.371711,0.0,0.0
74644,2018-05-09 23:59:30,1726-01-29 00:11:59.145224192,97.19,1.0,1.0,0.000012,1.0,2018-05-09,97.19,BID,...,1.0,1.184757e+07,11389.0,97.189069,0.005,3078.0,97.1875,0.627680,0.0,0.0


In [16]:
test2 = testYM[testYM['type']=='Trade']
test2.columns.values

array(['time', 'estimated_arrival_time', 'TradePrice', 'TradedVolume',
       'NoOfTrades', 'arrival_rate', 'AverageVolumePerTrade', 'date',
       'CCYTradedVolume', 'type', 'value', 'size', 'BestBid', 'BidSize',
       'BestAsk', 'AskSize', 'CumlTradingVolume', 'CCYCumlVolume',
       'CumlNoOfTrades', 'CumlTradedVolume', 'RunningNoOfTrades',
       'RunningCumlVolume', 'RunningCumlNoOfTrades', 'MicroPrice',
       'BASpread', 'QuoteSize', 'MidPrice', 'OrderImbalance',
       'ExpandingMeanRelativeTickVolume', 'MeanRelativeTickVolume'],
      dtype=object)

In [ ]:
test2 = test2[['time','TradePrice', 'TradedVolume','MicroPrice',
       'NoOfTrades', 'arrival_rate', 'AverageVolumePerTrade', 'date',
       'CCYTradedVolume', 'type', 'value', 'size', 'BestBid', 'BidSize',
       'BestAsk', 'AskSize', 'CumlTradingVolume', 'CCYCumlVolume',
       'CumlNoOfTrades', 'CumlTradedVolume', 'RunningNoOfTrades']]

In [18]:
def event_clock(df, clock_column, bar_size, *args):
    """
    This function resamples a dataframe using a custom event clock.
    df: DataFrame with necessary columns
    clock_column: string name of the column with the clock data
    bar_size: the cumulative sum for a bar in the clock
    *args: string names of the columns that you want to resample
    """
    df['clock'] = df[clock_column] // bar_size

    # For all columns not in *args, take the first value for each bar
    resampled_df = df.groupby('clock').first()

    # For columns in *args, compute OHLC values
    for arg in args:
        resampled_df[arg+'_open'] = df.groupby('clock')[arg].first()
        resampled_df[arg+'_high'] = df.groupby('clock')[arg].max()
        resampled_df[arg+'_low'] = df.groupby('clock')[arg].min()
        resampled_df[arg+'_close'] = df.groupby('clock')[arg].last()
    
    resampled_df['avg_best_bid_and_ask'] = (resampled_df['BestBid_close'] + resampled_df['BestAsk_close']) / 2

    # Remove the clock column from the original DataFrame
    df = df.drop(columns='clock')
    
    return resampled_df


In [19]:
import pandas as pd
import numpy as np

def resample_event_clock_one(df, clock_column, threshold, *args):
    """
    Resamples a DataFrame based on an event clock. Computes OHLC values for specified columns,
    and the median for the rest.

    Parameters:
    df (pd.DataFrame): The input dataframe
    clock_column (str): The column in df that acts as the event clock
    threshold (float): The threshold to trigger a new bar
    *args (str): Column names to calculate OHLC values for

    Returns:
    pd.DataFrame: A resampled dataframe with OHLC for specific columns and median for the rest
    """
    # Calculate a cumulative sum of the clock column
    df['clock_cumulative_sum'] = df[clock_column].cumsum()

    # Create a new column 'clock_bin' that increments every time 'clock_cumulative_sum' reaches the threshold
    df['clock_bin'] = df['clock_cumulative_sum'] // threshold

    # For columns in args, compute the OHLC values
    ohlc_df = pd.DataFrame()
    for arg in args:
        ohlc_df[arg + '_open'] = df.groupby('clock_bin')[arg].first()
        ohlc_df[arg + '_high'] = df.groupby('clock_bin')[arg].max()
        ohlc_df[arg + '_low'] = df.groupby('clock_bin')[arg].min()
        ohlc_df[arg + '_close'] = df.groupby('clock_bin')[arg].last()

    # For all other columns, compute the median
    median_df = df.drop(columns=list(args)).groupby('clock_bin').median()

    # Join the OHLC and median dataframes together and return
    resampled_df = ohlc_df.join(median_df)

    return resampled_df


In [20]:
def resample_event_clock_two(df, event_clock_column: str, 
                             event_clock_threshold: int, exclude_median: List[str], 
                             *args: str) -> pd.DataFrame:
    """
    Resamples a dataframe using an event-based clock.

    Parameters:
    df: pd.DataFrame
        The input dataframe to be resampled.
    event_clock_column: str
        The column name in df to be used as event clock.
    event_clock_threshold: int
        The threshold of cumulative number of events to define clock ticks.
    exclude_median: list
        List of column names for which median should not be computed.
    *args: str
        Columns for which first, max, min, last and median need to be computed.

    Returns:
    resampled_df: pd.DataFrame
        The resampled dataframe.
    """
    # Create event clock
    df['clockEvents'] = (df[event_clock_column].cumsum() // event_clock_threshold).astype(int)
    
    # Create OHLC dictionary
    ohlc_dict = {arg: ['first', 'max', 'min', 'last'] for arg in args}
    
    # Create median dictionary only for numeric columns
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    
    # Remove excluded columns from the list for median calculation
    median_cols = [col for col in numeric_cols if col not in exclude_median_columns]
    median_dict = {col: 'median' for col in median_cols}
    
    # Add event clock column to dictionary
    event_clock_dict = {event_clock_column: 'sum'}
    
    # Merge dictionaries
    agg_dict = {**ohlc_dict, **median_dict, **event_clock_dict}
    
    # Resample the dataframe
    resampled_df = df.groupby('clockEvents').agg(agg_dict)
        
#     # Reset the index
#     resampled_df.reset_index(inplace=True)
    
#     # Rename 'clock' column
#     resampled_df.rename(columns={'clock': event_clock_column + '_clock'}, inplace=True)

    
    return resampled_df


In [21]:
from typing import List
import numpy as np
import pandas as pd

def resample_event_clock_final(df: pd.DataFrame, event_clock_column: str, 
                             event_clock_threshold: int, exclude_median: List[str], 
                             *args: str) -> pd.DataFrame:
    """
    Resamples a dataframe using an event-based clock.

    Parameters:
    df: pd.DataFrame
        The input dataframe to be resampled.
    event_clock_column: str
        The column name in df to be used as event clock.
    event_clock_threshold: int
        The threshold of cumulative number of events to define clock ticks.
    exclude_median: list
        List of column names for which median should not be computed.
    *args: str
        Columns for which first, max, min, last and median need to be computed.

    Returns:
    resampled_df: pd.DataFrame
        The resampled dataframe.
    """

    # Create event clock, using a copy of the dataframe to avoid side-effects
    df = df.copy()
    df['clockEvents'] = (df[event_clock_column].cumsum() // event_clock_threshold).astype(int)
    
    # Create OHLC dictionary
    ohlc_dict = {arg: ['first', 'max', 'min', 'last'] for arg in args if arg in df.columns}
    
    # Create median dictionary only for numeric columns
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    
    # Remove excluded columns from the list for median calculation
    median_cols = [col for col in numeric_cols if col not in exclude_median and col in df.columns]
    median_dict = {col: 'median' for col in median_cols}
    
    # Add event clock column to dictionary
    event_clock_dict = {event_clock_column: 'sum'}
    
    # Merge dictionaries
    agg_dict = {**ohlc_dict, **median_dict, **event_clock_dict}
    
    # Resample the dataframe, compute directly on groupby result for improved performance
    resampled_df = df.groupby('clockEvents', as_index=False).agg(agg_dict)
    
    # Rename 'clock' column
    resampled_df.rename(columns={'clockEvents': event_clock_column + '_clock'}, inplace=True)

    return resampled_df


In [22]:
from typing import List

def event_clock_resample(df: pd.DataFrame, event_clock_column: str, 
                             volume_threshold: int, exclude_median: List[str], 
                             *args: str) -> pd.DataFrame:
    """
    Resamples a dataframe using an event-based clock.

    Parameters:
    df: pd.DataFrame
        The input dataframe to be resampled.
    event_clock_column: str
        The column name in df to be used as event clock.
    volume_threshold: int
        The threshold of cumulative volume to define clock ticks.
    exclude_median: list
        List of column names for which median should not be computed.
    *args: str
        Columns for which first, max, min, last and median need to be computed.

    Returns:
    resampled_df: pd.DataFrame
        The resampled dataframe.
    """
    df['cum_vol'] = df[event_clock_column].cumsum()
    df['clock'] = (df['cum_vol'] / volume_threshold).astype(int)

    ohlc_dict = {arg: ['first', 'max', 'min', 'last'] for arg in args if arg not in exclude_median}
    median_dict = {arg: 'median' for arg in args if arg not in exclude_median}
    volume_dict = {event_clock_column: 'sum'}

    agg_dict = {**ohlc_dict, **median_dict, **volume_dict}

    resampled_df = df.groupby('clock').agg(agg_dict)

    # Reset the index
    resampled_df.reset_index(inplace=True)

    # Rename 'clock' column
    resampled_df.rename(columns={'clock': event_clock_column + '_clock'}, inplace=True)

    return resampled_df


In [31]:
testYM.head(10)

,time,estimated_arrival_time,TradePrice,TradedVolume,NoOfTrades,arrival_rate,AverageVolumePerTrade,date,CCYTradedVolume,type,...,RunningNoOfTrades,RunningCumlVolume,RunningCumlNoOfTrades,MicroPrice,BASpread,QuoteSize,MidPrice,OrderImbalance,ExpandingMeanRelativeTickVolume,MeanRelativeTickVolume
1055839,2018-05-07 00:00:20,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,ASK,...,1.0,1697079.24,424.0,97.801802,0.005,10043.0,97.8025,-0.279299,NaN,0.0
1058172,2018-05-07 00:00:51,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,BID,...,1.0,1697079.24,424.0,97.801094,0.005,8223.0,97.8025,-0.562447,NaN,0.0
1055840,2018-05-07 00:01:03,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,ASK,...,1.0,1697079.24,424.0,97.801799,0.005,5001.0,97.8025,-0.280544,NaN,0.0
1055841,2018-05-07 00:01:31,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,ASK,...,1.0,1697079.24,424.0,97.801791,0.005,5021.0,97.8025,-0.283410,NaN,0.0
1058173,2018-05-07 00:01:31,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,BID,...,1.0,1697079.24,424.0,97.803882,0.005,14416.0,97.8025,0.552997,NaN,0.0
1058174,2018-05-07 00:01:32,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,BID,...,1.0,1697079.24,424.0,97.801790,0.005,5018.0,97.8025,-0.284177,NaN,0.0
1058175,2018-05-07 00:01:36,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,BID,...,1.0,1697079.24,424.0,97.801791,0.005,5021.0,97.8025,-0.283410,NaN,0.0
1058176,2018-05-07 00:01:41,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,BID,...,1.0,1697079.24,424.0,97.801790,0.005,5018.0,97.8025,-0.284177,NaN,0.0
1055842,2018-05-07 00:01:49,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,ASK,...,1.0,1697079.24,424.0,97.801903,0.005,4718.0,97.8025,-0.238660,NaN,0.0
1055843,2018-05-07 00:01:54,NaT,NaN,NaN,NaN,NaN,NaN,2018-05-07,NaN,ASK,...,1.0,1697079.24,424.0,97.801408,0.005,6380.0,97.8025,-0.436991,NaN,0.0


In [25]:
# Define the columns that you want to compute the OHLC values for
columns_to_resample = ['BestBid', 'BestAsk','MicroPrice']
exclude_median_columns =['time']
# Call the function
resampled_df_1 = resample_event_clock_two(test2, 'NoOfTrades', 1, *columns_to_resample)
resampled_df_2 = resample_event_clock_two(test2, 'NoOfTrades', 30, *columns_to_resample)
resampled_df_3 = resample_event_clock_two(test2, 'TradedVolume', 50, *columns_to_resample)
resampled_df_4 = resample_event_clock_two(test2, 'CumlTradingVolume', 250, *columns_to_resample)



/home/ak/anaconda3/envs/timeSeries/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [26]:
resampled_df_1

,BestAsk,MicroPrice,TradePrice,TradedVolume,NoOfTrades,arrival_rate,AverageVolumePerTrade,CCYTradedVolume,value,size,...,RunningNoOfTrades,RunningCumlVolume,RunningCumlNoOfTrades,BASpread,QuoteSize,MidPrice,OrderImbalance,ExpandingMeanRelativeTickVolume,MeanRelativeTickVolume,clockEvents
clockEvents,,,,,,,,,,,,,,,,,,,,,
1,97.805,97.801183,97.800,2.0,1.0,0.000012,2.000000,195.600,97.800,1814.0,...,1.0,195.600,1.0,0.005,7666.0,97.8025,-0.526741,24.000000,0.133178,1.0
3,97.805,97.800926,97.805,8.0,2.0,0.000023,4.000000,782.440,97.800,1974.0,...,2.0,978.040,3.0,0.005,10653.0,97.8025,-0.629400,22.400000,0.532712,3.0
25,97.805,97.802556,97.805,932.0,22.0,0.000255,42.363636,91154.260,97.805,1934.0,...,22.0,92132.300,25.0,0.005,3956.0,97.8025,0.022245,40.564756,62.060946,25.0
26,97.810,97.806945,97.805,7.0,1.0,0.000012,7.000000,684.635,97.805,1910.0,...,1.0,92816.935,26.0,0.005,4910.0,97.8075,-0.221996,0.486828,0.466123,26.0
27,97.810,97.806384,97.805,7.0,1.0,0.000012,7.000000,684.635,97.805,2787.0,...,1.0,93501.570,27.0,0.005,10067.0,97.8075,-0.446310,0.600418,0.466123,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,97.815,97.811426,97.810,28.0,1.0,0.000012,28.000000,2738.680,97.815,3097.0,...,1.0,7841549.115,2191.0,0.005,4333.0,97.8125,-0.429495,1.897050,1.864492,2191.0
2228,97.815,97.808079,97.810,2058.0,37.0,0.000429,55.621622,201292.980,97.815,51173.0,...,37.0,8042842.095,2228.0,0.010,73940.0,97.8100,-0.384176,136.218963,137.040157,2228.0
2230,97.815,97.810016,97.810,165.0,2.0,0.000023,82.500000,16138.650,97.810,167.0,...,2.0,8058980.745,2230.0,0.005,51340.0,97.8125,-0.993494,10.905483,10.987185,2230.0


In [27]:
# Define the columns that you want to compute the OHLC values for
columns_to_resample = ['BestBid', 'BestAsk','MicroPrice']
exclude_median_columns =['time']
# Call the function
final_resampled_df_1 = resample_event_clock_final(test2, 'NoOfTrades', 1, *columns_to_resample)
final_resampled_df_2 = resample_event_clock_final(test2, 'NoOfTrades', 30, *columns_to_resample)
final_resampled_df_3 = resample_event_clock_final(test2, 'TradedVolume', 50, *columns_to_resample)
final_resampled_df_4 = resample_event_clock_final(test2, 'CumlTradingVolume', 250, *columns_to_resample)


In [ ]:
resampled_df_4.head(5)

In [28]:
final_resampled_df_4.columns.values

array(['BestAsk', 'MicroPrice', 'TradePrice', 'TradedVolume',
       'NoOfTrades', 'arrival_rate', 'AverageVolumePerTrade',
       'CCYTradedVolume', 'value', 'size', 'BidSize', 'AskSize',
       'CumlTradingVolume', 'CCYCumlVolume', 'CumlNoOfTrades',
       'CumlTradedVolume', 'RunningNoOfTrades', 'RunningCumlVolume',
       'RunningCumlNoOfTrades', 'BASpread', 'QuoteSize', 'MidPrice',
       'OrderImbalance', 'ExpandingMeanRelativeTickVolume',
       'MeanRelativeTickVolume', 'CumlTradingVolume_clock'], dtype=object)

In [ ]:
resampled_df_2

In [ ]:
resampled_df_4.head(5)

In [29]:
# Call the function
resampled_df_1 = event_clock_resample(test2, 'NoOfTrades', 1, *columns_to_resample)
resampled_df_2 = event_clock_resample(test2, 'NoOfTrades', 10, *columns_to_resample)
resampled_df_3 = event_clock_resample(test2, 'TradedVolume', 50, *columns_to_resample)
resampled_df_4 = event_clock_resample(test2, 'CCYTradedVolume', 250, *columns_to_resample)


/home/ak/anaconda3/envs/timeSeries/lib/python3.7/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [30]:
resampled_df_1

,NoOfTrades_clock,BestAsk,MicroPrice,NoOfTrades
0,1,97.805,97.801183,1.0
1,3,97.805,97.800926,2.0
2,25,97.805,97.802556,22.0
3,26,97.810,97.806945,1.0
4,27,97.810,97.806384,1.0
...,...,...,...,...
599,2191,97.815,97.811426,1.0
600,2228,97.815,97.808079,37.0
601,2230,97.815,97.810016,2.0
602,2231,97.810,97.808694,1.0


In [ ]:
EventClocks ='/media/ak/Data/InterestRateFuturesData/EventClocksFiles'

In [ ]:
import os

def create_nested_directories(base_path,symbols_set1, symbols_set2):
    
    for symbol1 in symbols_set1:
        dir1 = os.path.join(base_path, symbol1)
        os.makedirs(dir1, exist_ok=True)

        for symbol2 in symbols_set2:
            dir2 = os.path.join(dir1, symbol2)
            os.makedirs(dir2, exist_ok=True)

In [ ]:
symbols_set1 = ['DU1', 'FB1', 'FV1', 'G', 'JB1', 'KE1', 'OE1', 'RX1', 'US1', 'US2y']
symbols_set2 =['tick','dollar','volume']

In [ ]:
create_nested_directories(EventClocks, symbols_set1, symbols_set2)

In [ ]:
files = os.listdir(debug)

In [ ]:
filedf = pd.read_pickle(file)

In [ ]:
filedf.columns.values

In [ ]:
filedf

In [ ]:
debug ='/media/ak/Data/InterestRateFuturesData/EventClocksFiles/XM1/tick'

In [ ]:
file = os.path.join(debug, files[9])

In [ ]:
pd.read_pickle(file)